
# quickdraw-doodle-recognition

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import dill as pickle

/home/borowis/.pyenv/versions/3.6.6/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH = '/home/borowis/qdr/s3'

In [4]:
ls {PATH}

cat2idx.pkl  idx2cat.pkl  sample_submission.csv  tmp/
data/        models/      test/                  train/


In [5]:
bs = 256
BASE_SIZE = 256                  # image size
NUM_SAMPLES_PER_CLASS = 10       # set to -1 for all images
CHUNK_SIZE = 5000                # csv processing chunk

# csv data to images

In [6]:
def draw_cv2(raw_strokes, size=256, lw=2, time_color=True):
    colors = [(255, 0, 0) , (255, 255, 0),  (128, 255, 0),  (0, 255, 0), (0, 255, 128), (0, 255, 255), 
              (0, 128, 255), (0, 0, 255), (128, 0, 255), (255, 0, 255)]
    img = np.zeros((BASE_SIZE, BASE_SIZE, 3), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = colors[min(t, len(colors)-1)]
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw, lineType=cv2.LINE_4)
    img = img.astype(np.float32)/255
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

In [7]:
def save_array(name, array):
    os.makedirs(f'{PATH}/data/{name}', exist_ok=True)
    barray = bcolz.carray(array, rootdir=f'{PATH}/data/{name}', mode='w')
    barray.flush()
    return barray
    
def save_array_from_series(name, series): 
    os.makedirs(f'{PATH}/data/{name}', exist_ok=True)
    barray = bcolz.carray(np.ndarray(shape=(0, BASE_SIZE, BASE_SIZE, 3), dtype=np.float32), expectedlen=len(series), 
                          rootdir=f'{PATH}/data/{name}', mode='w')
    
    for start in range(0, len(series), CHUNK_SIZE):
        chunk = series[start:start+CHUNK_SIZE].apply(json.loads).apply(draw_cv2).values
        barray.append(np.stack(chunk))

    barray.flush()
    return barray
    
def load_array(name):
    return bcolz.open(rootdir=f'{PATH}/data/{name}')

In [50]:
df = pd.read_csv(f'{PATH}/train/airplane.csv')

In [52]:
len(df)

151623

In [ ]:
img = draw_cv2(json.loads(df.iloc[0]['drawing']))
#plt.imshow(img)
#plt.savefig(f'{PATH}/train/images/{df.iloc[0]["key_id"]}.png')
cv2.imwrite(f'{PATH}/train/images/{df.iloc[0]["key_id"]}.webp', img)

In [ ]:
#plt.imshow(plt.imread(f'{PATH}/train/images/{df.iloc[0]["key_id"]}.webp'))
plt.imshow(df.iloc[:5]['drawing'].apply(json.loads).apply(draw_cv2)[0])

In [ ]:
categories = [fileName.replace('.csv', '') for fileName in os.listdir(f'{PATH}/train') if ('.csv' in fileName)]
categories2idx = {c:idx for idx, c in enumerate(categories)}
idx2category = {idx:c for idx, c in enumerate(categories)}

In [8]:
#pickle.dump(categories2idx, open(f'{PATH}/cat2idx.pkl', 'wb'))
#pickle.dump(idx2category, open(f'{PATH}/idx2cat.pkl', 'wb'))

categories2idx = pickle.load(open(f'{PATH}/cat2idx.pkl', 'rb'))
idx2category = pickle.load(open(f'{PATH}/idx2cat.pkl', 'rb'))
categories = [k for k in categories2idx.keys()]

In [ ]:
os.makedirs(f'{PATH}/data/X_val', exist_ok=True)
os.makedirs(f'{PATH}/data/Y_val', exist_ok=True)
os.makedirs(f'{PATH}/data/X_train', exist_ok=True)
os.makedirs(f'{PATH}/data/Y_train', exist_ok=True)

X_val = bcolz.carray(np.ndarray(shape=(0, BASE_SIZE, BASE_SIZE, 3), dtype=np.float32), expectedlen=50000*300, 
                     rootdir=f'{PATH}/data/X_val', mode='w')
X_train = bcolz.carray(np.ndarray(shape=(0, BASE_SIZE, BASE_SIZE, 3), dtype=np.float32), expectedlen=150000*300, 
                       rootdir=f'{PATH}/data/X_train', mode='w')

Y_val = bcolz.carray(np.array([], dtype=np.float32), expectedlen=50000*300, rootdir=f'{PATH}/data/Y_val', mode='w')
Y_train = bcolz.carray(np.array([], dtype=np.float32), expectedlen=150000*300, rootdir=f'{PATH}/data/Y_train', mode='w')

for category in categories:
    df = pd.read_csv(f'{PATH}/train/{category}.csv')
    
    for start in range(0, len(df), CHUNK_SIZE):
        chunk = np.stack(df['drawing'][start:start+CHUNK_SIZE].apply(json.loads).apply(draw_cv2).values)
        chunky = np.full(len(chunk), categories2idx[category])
        
        ((chunk_val, chunk_train), (chunky_val, chunky_train)) = split_by_idx(
            get_cv_idxs(len(chunk), seed=None), chunk, chunky)
        
        X_val.append(chunk_val)
        Y_val.append(chunky_val)
        
        X_train.append(chunk_train)
        Y_train.append(chunky_train)
        
    X_train.flush()
    X_val.flush()
    
    Y_train.flush()
    Y_val.flush()

In [48]:
#save_array('X_train', X_train)
#save_array('X_val', X_val)
#save_array('Y_train', Y_train)
#save_array('Y_val', Y_val)

X_train = load_array('X_train')
X_val = load_array('X_val')
Y_train = load_array('Y_train')
Y_val = load_array('Y_val')

In [15]:
df_test = pd.read_csv(f'{PATH}/test/test_simplified.csv')

In [9]:
# X_test = save_array_from_series('X_test', df_test['drawing'])
X_test = load_array('X_test')

In [10]:
# Y_test = save_array('Y_test', df_test['key_id'].astype(str).values)
Y_test = load_array('Y_test')

In [ ]:
X_train

# multi label

In [ ]:
metrics = [accuracy]
f_model = resnet34

In [ ]:
def get_data(sz):
    tfms = tfms_from_model(f_model, sz, transforms_basic, max_zoom = 1.1)
    return ImageClassifierData.from_arrays(PATH, (X_train, Y_train), (X_val, Y_val), bs=bs, tfms = tfms)

In [ ]:
data = get_data(256)
x, y = next(iter(data.val_dl))

## size 64

In [ ]:
data = get_data(64)

In [ ]:
learn = ConvLearner.pretrained(f_model, data, metrics = metrics)

In [ ]:
lrf = learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 0.2

In [ ]:
learn.fit(lr, 3, cycle_len = 1, cycle_mult = 2)

In [ ]:
lrs = np.array([lr / 9, lr / 3, lr])
learn.unfreeze()

In [ ]:
learn.fit(lrs, 3, cycle_len = 1, cycle_mult = 2)

In [ ]:
# learn.save('64')
learn.load('64')

In [ ]:
learn.sched.plot_loss()

## size 128

In [ ]:
learn.set_data(get_data(128))
learn.freeze()
learn.fit(lr, 3, cycle_len = 1, cycle_mult = 2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len = 1, cycle_mult = 2)

In [ ]:
# learn.save('128')
learn.load('128')

In [ ]:
learn.sched.plot_loss()

## size 256

In [ ]:
learn.set_data(get_data(256))
learn.freeze()
learn.fit(lr, 3, cycle_len = 1, cycle_mult = 2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len = 1, cycle_mult = 2)

In [ ]:
# learn.save('256')
learn.load('256')

In [ ]:
learn.sched.plot_loss()

## end

In [ ]:
log_preds, y = learn.TTA()

In [ ]:
preds = np.mean(log_preds, axis = 0)
f2(preds, y)

## submission

In [ ]:
val = learn.predict()

In [ ]:
f2(val,data.val_y)

In [ ]:
log_preds = learn.TTA(is_test=True)

In [ ]:
th = opt_th(preds, y); th

In [ ]:
preds = np.mean(log_preds[0], axis=0)
cls = np.array(data.classes)
res = np.array([" ".join(cls[(np.where(pp > th))]) for pp in preds])

In [ ]:
fnames = [f[9:-4] for f in data.test_dl.dataset.fnames]

In [ ]:
outp = pd.DataFrame({'image_name': fnames, 'tags': res})
outp.head()